In [34]:
import wandb
import pandas as pd 
import os 
import json
api = wandb.Api()

In [35]:
data = {}
for project in api.projects():
    if 'Letter-high' in project.name and 'reg_repeat' in project.name:
        runs = api.runs(os.path.join(project.path[0], project.path[1]))
        msg = project.path[1].split('_')
        model_name = msg[0]
        dataset_name = msg[1]
        
        if model_name not in data:
            data[model_name] = {}

        if dataset_name not in data[model_name]:
            data[model_name][dataset_name] = []
                
        for run in runs:
            try:
                data[model_name][dataset_name].append(run.summary['test_acc_of_early_stop'])
            except:
                continue

In [41]:
import numpy as np 
print(np.mean(data['GCN']['Letter-high'][:10]))
print(data['GCN']['Letter-high'][:10])
print(np.std(data['GCN']['Letter-high'][:10]))

0.5817777777777777
[0.5955555555555555, 0.5911111111111111, 0.6, 0.5644444444444444, 0.5777777777777777, 0.5244444444444445, 0.5911111111111111, 0.5866666666666667, 0.6044444444444445, 0.5822222222222222]
0.02199438761183754


In [25]:
import numpy as np
for key in data:
    d = np.array(data[key]['Letter-high'])
    print(f'{key}: {np.mean(d)}±{np.std(d)}')

GMT: 0.5155555555555555±0.05130398391384532
GCN: 0.5768888888888889±0.029265915792274375
GIN: 0.6777777777777778±0.044322053702919105
GraphSAGE: 0.8453333333333333±0.023247461032216924
GTransformer: 0.8795555555555555±0.015956731618681125


In [3]:
for project in api.projects():
    if project.name.endswith("_model"):
        runs = api.runs(os.path.join(project.path[0], project.path[1]))
        for run in runs:
            run.delete()
        
    

In [ ]:
import wandb

# Initialize the wandb API (you might need to log in first)
api = wandb.Api()

# List all projects in your account
projects = api.projects()

# Filter projects that end with "basemode"
basemode_projects = [proj for proj in projects if proj.name.endswith("basemodel")]

# Read the config of each "basemode" project
for project in basemode_projects:
    print(f"Config for project {project.name}:")
    
    runs = api.runs(path=project.path[1])
    for run in runs:
        print(run.config.get("batch_size"), run.config.get("dropout"), run.config.get("epoch"), run.config.get("hidden_size"), run.config.get("learning_rate"), run.config.get("num_layer"), run.config.get("weight_decay"))
        break
    print("\n")


In [ ]:
data = {}
for project in api.projects():
    if project.path[1].endswith("_model"):
        # print(project.path[1])
        runs = api.runs(os.path.join(project.path[0], project.path[1]))
        msg = project.path[1].split('_')
        model_name = msg[0]
        dataset_name = msg[1]
        
        if model_name not in data:
            data[model_name] = {}

        if dataset_name not in data[model_name]:
            data[model_name][dataset_name] = []
                
        for run in runs:
            try:
                data[model_name][dataset_name].append(run.summary['test_acc_of_early_stop'])
            except:
                continue
        

In [ ]:
model_name_list = list(data.keys())
dataset_name_list = list(data[model_name_list[1]].keys())

In [ ]:
import numpy as np
frame = [[''] * len(dataset_name_list) for _ in range(len(model_name_list))]

for i,m in enumerate(model_name_list):
    if m == "PPGN":
        continue
    if m not in data:
        continue

    for j,d in enumerate(dataset_name_list):
        if d not in data[m]:
            continue
        acc = np.array(data[m][d])
        mean = str(round(acc.mean(), 3))
        diff = str(round(np.std(acc), 4))
        item = mean+"±"+diff
        frame[i][j] = item

data_df = pd.DataFrame(frame, index=model_name_list, columns=dataset_name_list)
print(data_df)

                 NCI109          NCI1      PROTEINS
GraphSage   0.754±0.012  0.781±0.0196  0.651±0.0862
MixHop     0.734±0.0107  0.774±0.0103  0.559±0.0878
GAT        0.725±0.0097  0.763±0.0179  0.735±0.0167
GCN        0.738±0.0104  0.762±0.0101   0.742±0.021
GIN        0.759±0.0101  0.792±0.0134  0.691±0.0369


0.75086